In [4]:
# Instala as ferramentas necessárias para gerar e trabalhar com os dados
!pip install faker
!pip install pandas
!pip install numpy
!pip install google-cloud-bigquery
!pip install gcsfs

print("Instalação das ferramentas concluída!")


Instalação das ferramentas concluída!


In [5]:
# Importa as ferramentas que acabamos de instalar
import pandas as pd
import numpy as np
from faker import Faker
from datetime import datetime, timedelta
import random

# Vamos criar um gerador de dados falsos em português do Brasil
fake = Faker('pt_BR')
Faker.seed(42) # Semente para garantir que os dados sejam sempre os mesmos

# ----------------------------------------------------------------------
# 1. LOTES (A Tabela Principal de Áreas da Obra)
# ----------------------------------------------------------------------
num_lots = 5
lot_data = []
lots_ids = [f'LOTE-{i:02d}' for i in range(1, num_lots + 1)]
start_date = datetime(2023, 1, 1)

for lot_id in lots_ids:
    base_cost = random.randint(150_000_000, 300_000_000)
    base_duration = random.randint(300, 450) # Duração em dias

    lot_data.append({
        'lot_id': lot_id,
        'location': f'Setor {random.choice(["Norte", "Sul", "Leste", "Oeste"])} - KM {random.randint(100, 900)}',
        'scheduled_start': start_date + timedelta(days=random.randint(0, 30)),
        'scheduled_end': start_date + timedelta(days=base_duration + random.randint(-20, 20)),
        'total_budget_BRL': base_cost,
        'km_extent': random.randint(50, 150)
    })

df_lots = pd.DataFrame(lot_data)


# ----------------------------------------------------------------------
# 2. RELATÓRIOS DE PROGRESSO (O que foi feito até hoje)
# ----------------------------------------------------------------------
current_date = datetime(2023, 1, 1)
num_days = 100
progress_data = []

for day in range(num_days):
    for lot_id in lots_ids:
        daily_increase = random.uniform(0.1, 0.5)
        previous_progress = next((p['progress_percentage'] for p in progress_data if p['lot_id'] == lot_id and p['report_date'] == current_date - timedelta(days=1)), 0.0)
        new_progress = min(100.0, previous_progress + daily_increase)

        if random.random() < 0.10:
            delay_days = random.randint(1, 5)
        else:
            delay_days = 0

        progress_data.append({
            'report_date': current_date,
            'lot_id': lot_id,
            'progress_percentage': new_progress,
            'km_completed': (new_progress / 100) * df_lots[df_lots['lot_id'] == lot_id]['km_extent'].iloc[0],
            'estimated_delay_days': delay_days,
            'weather_conditions': random.choice(['Sol Forte', 'Nublado', 'Chuva Leve', 'Chuva Intensa']),
        })
    current_date += timedelta(days=1)

df_progress = pd.DataFrame(progress_data)
df_progress['progress_percentage'] = df_progress['progress_percentage'].round(2)
df_progress['km_completed'] = df_progress['km_completed'].round(2)


# ----------------------------------------------------------------------
# 3. CUSTOS (O dinheiro que gastamos)
# ----------------------------------------------------------------------
num_cost_entries = 500
cost_data = []

for _ in range(num_cost_entries):
    cost_data.append({
        'cost_id': fake.uuid4(),
        'lot_id': random.choice(lots_ids),
        'date': fake.date_between(start_date=start_date, end_date=current_date - timedelta(days=1)),
        'cost_type': random.choice(['Salários', 'Combustível', 'Materiais Brutos', 'Aluguel de Máquinas', 'Manutenção']),
        'amount_BRL': random.randint(10000, 500000)
    })

df_costs = pd.DataFrame(cost_data)


# ----------------------------------------------------------------------
# 4. INCIDENTES (Problemas e acidentes)
# ----------------------------------------------------------------------
num_incidents = 150
incident_data = []

for _ in range(num_incidents):
    incident_data.append({
        'incident_id': fake.uuid4(),
        'lot_id': random.choice(lots_ids),
        'date': fake.date_between(start_date=start_date, end_date=current_date - timedelta(days=1)),
        'severity': random.choice(['Baixa', 'Média', 'Alta', 'Crítica']),
        'type': random.choice(['Atraso no Suprimento', 'Falha de Máquina', 'Acidente Leve', 'Acidente Grave', 'Paralisação por Clima']),
        'downtime_hours': random.uniform(0.5, 8.0)
    })

df_incidents = pd.DataFrame(incident_data)
df_incidents['downtime_hours'] = df_incidents['downtime_hours'].round(1)


# ----------------------------------------------------------------------
# 5. REGISTROS DE SUPRIMENTOS (O que usamos)
# ----------------------------------------------------------------------
num_supply_entries = 300
supply_data = []

for _ in range(num_supply_entries):
    supply_data.append({
        'supply_id': fake.uuid4(),
        'lot_id': random.choice(lots_ids),
        'date': fake.date_between(start_date=start_date, end_date=current_date - timedelta(days=1)),
        'item': random.choice(['Trilhos de Aço', 'Dormentes', 'Lastro (Brita)', 'Combustível Diesel', 'Cimento']),
        'quantity': random.randint(100, 5000),
        'unit': random.choice(['toneladas', 'unidades', 'litros', 'm³']),
    })

df_supply = pd.DataFrame(supply_data)


# ----------------------------------------------------------------------
# 6. INSTANTÂNEOS DE SATÉLITE (Como a obra parecia em certos dias)
# ----------------------------------------------------------------------
num_snapshots = 50
snapshot_data = []

for _ in range(num_snapshots):
    snapshot_data.append({
        'snapshot_id': fake.uuid4(),
        'lot_id': random.choice(lots_ids),
        'date': fake.date_between(start_date=start_date, end_date=current_date - timedelta(days=1)),
        'area_cleared_sqm': random.randint(50000, 500000),
        'equipment_count': random.randint(5, 30)
    })

df_satellite_snapshots = pd.DataFrame(snapshot_data)


# ----------------------------------------------------------------------
# 7. FUNCIONÁRIOS (Dados simples sobre a equipe)
# ----------------------------------------------------------------------
num_employees = 100
employee_data = []

for i in range(1, num_employees + 1):
    employee_data.append({
        'employee_id': f'EMP-{i:03d}',
        'lot_id': random.choice(lots_ids),
        'name': fake.name(),
        'role': random.choice(['Engenheiro Civil', 'Operador de Máquina', 'Técnico de Segurança', 'Auxiliar Geral', 'Mestre de Obras']),
        'contract_type': random.choice(['CLT', 'Terceirizado']),
        'salary_BRL': random.randint(2500, 15000)
    })

df_employees = pd.DataFrame(employee_data)


# ----------------------------------------------------------------------
# 8. SALVANDO OS ARQUIVOS (Onde o robô transforma dados em planilhas)
# ----------------------------------------------------------------------
print("Salvando as planilhas (CSVs)...")

df_lots.to_csv('lots.csv', index=False)
df_progress.to_csv('progress.csv', index=False)
df_costs.to_csv('costs.csv', index=False)
df_incidents.to_csv('incidents.csv', index=False)
df_supply.to_csv('supply.csv', index=False)
df_satellite_snapshots.to_csv('satellite_snapshots.csv', index=False)
df_employees.to_csv('employees.csv', index=False)

print("SUCESSO! 7 arquivos CSVs criados e salvos!")

Salvando as planilhas (CSVs)...
SUCESSO! 7 arquivos CSVs criados e salvos!


In [6]:
import pandas as pd
import numpy as np

print("Iniciando a Limpeza e Preparação dos 7 arquivos...")

# ----------------------------------------------------------------------
# 1. CARREGAR OS DADOS (Lendo os arquivos CRUS que acabamos de criar)
# ----------------------------------------------------------------------
df_lots = pd.read_csv('lots.csv')
df_progress = pd.read_csv('progress.csv')
df_costs = pd.read_csv('costs.csv')
df_incidents = pd.read_csv('incidents.csv')
df_supply = pd.read_csv('supply.csv')
df_satellite_snapshots = pd.read_csv('satellite_snapshots.csv')
df_employees = pd.read_csv('employees.csv')

# ----------------------------------------------------------------------
# 2. LIMPEZA E TRANSFORMAÇÃO GERAL (Arrumando o formato das colunas)
# ----------------------------------------------------------------------

# TRATANDO DATAS
df_lots['scheduled_start'] = pd.to_datetime(df_lots['scheduled_start'])
df_lots['scheduled_end'] = pd.to_datetime(df_lots['scheduled_end'])
df_progress['report_date'] = pd.to_datetime(df_progress['report_date'])
df_costs['date'] = pd.to_datetime(df_costs['date'])
df_incidents['date'] = pd.to_datetime(df_incidents['date'])
df_supply['date'] = pd.to_datetime(df_supply['date'])
df_satellite_snapshots['date'] = pd.to_datetime(df_satellite_snapshots['date'])

# TRATANDO CUSTOS
df_costs['amount_BRL'] = df_costs['amount_BRL'].round(2)


# ----------------------------------------------------------------------
# 3. CRIAÇÃO DE NOVAS MÉTRICAS
# ----------------------------------------------------------------------

# Gasto Acumulado por Lote e por Tipo
df_costs_summary = df_costs.groupby(['lot_id', 'cost_type']).agg(
    total_spent_BRL=('amount_BRL', 'sum')
).reset_index()

# Placar de Criticidade
df_incidents['criticality_score'] = df_incidents['severity'].map({
    'Baixa': 1,
    'Média': 2,
    'Alta': 3,
    'Crítica': 5
})


# ----------------------------------------------------------------------
# 4. TRATAMENTO DE VALORES FALTANTES
# ----------------------------------------------------------------------
df_incidents['downtime_hours'] = df_incidents['downtime_hours'].fillna(0)
df_progress['estimated_delay_days'] = df_progress['estimated_delay_days'].apply(lambda x: max(0, x))

print("Limpeza e Transformação Concluídas.")


# ----------------------------------------------------------------------
# 5. SALVAR OS ARQUIVOS LIMPOS COM NOVO NOME (*_clean.csv)
# ----------------------------------------------------------------------

print("Salvando as Planilhas LIMPAS (*_clean.csv)...")

# Salvamos com '_clean' no final para sabermos que estão prontos para o BigQuery
df_lots.to_csv('lots_clean.csv', index=False)
df_progress.to_csv('progress_clean.csv', index=False)
df_costs.to_csv('costs_clean.csv', index=False)
df_incidents.to_csv('incidents_clean.csv', index=False)
df_supply.to_csv('supply_clean.csv', index=False)
df_satellite_snapshots.to_csv('satellite_snapshots_clean.csv', index=False)
df_employees.to_csv('employees_clean.csv', index=False)

print("SUCESSO! 7 arquivos *_clean.csv prontos para o BigQuery!")

Iniciando a Limpeza e Preparação dos 7 arquivos...
Limpeza e Transformação Concluídas.
Salvando as Planilhas LIMPAS (*_clean.csv)...
SUCESSO! 7 arquivos *_clean.csv prontos para o BigQuery!
